# This is Dave's Movie Recommender
> Type in your favorite movie and the model will return a list of similar movies.

In [ ]:
#| default_exp movie_recommender

[Fastai](https://youtu.be/p4ZZq0736Po?t=3724) is the library we use to train, load and run our model.  NB: we already trained our model so we'll only be loading it here.

In [ ]:
1+1

2

In [ ]:
# !pip install -Uqq fastai 

[Gradio](https://gradio.app/getting_started/) allows you to easily publish your model onto the web for peope to use.

In [ ]:
# !pip install gradio

[nbdev](https://nbdev.fast.ai/tutorials/tutorial.html) is what we're using to make this documentation easier and more presentable.

In [ ]:
# !pip install nbdev

In [ ]:
# For modeling
from fastai.tabular.all import *
from fastai.collab import *
#for publishing the model
import gradio as gr

In [ ]:
#| hide
path = Path('/Users/davidramsey/Documents/coding/mov12/models_and_dls')

In [ ]:
#load your the model and data loaders
learn = load_learner(path/'model_movie_predictor_large9mil_ds_19_062.pkl')
dls = torch.load(path/'movie_9mil_03_dls')

In [ ]:
#| export
def get_movie_recs(full_title):
    "This function will use the `nn.CosineSimilarity` on the 50 latent factors for each movie to find the 30 movies most similar to your favorite movie."
    movie_factors = learn.model.i_weight.weight
    idx = dls.classes['title'].o2i[full_title]
    distances =  nn.CosineSimilarity(dim=1)(movie_factors,movie_factors[idx][None])
    idx = distances.argsort(descending=True)
    return [o for o in dls.classes['title'][idx][:30]]

In [ ]:
#| export
def search_movies(favorite_movie):
    movies_found = ''
    s = favorite_movie.lower()
    #remove THE from the title
    if s[:4] == 'the ':
        s = s[4:]

    lst = dls.classes['title']
    index = []
    i=0
    length = len(lst)
    while i<length:
        if s in lst[i].lower():
            full_title = lst[i]
            movies_found+= str(full_title) +'\n'
#             print(f'Your Favorite Movies:  {full_title}')
        i+=1
    # write explaination in case multiple movies
    explainer = f'If there are multiple movies above: Please paste your favorite movie into the "favorite_movie" box.\n The box below is currently showing recommendations for the movie:  {full_title}'
    #print movies found and explainer
    output_str =  movies_found+'\n\n'+explainer
    
    #get recommendations from model
    recommendations = get_movie_recs(full_title) 
    #create list of all the recommendations to print
    print_lst = ''
    for o in recommendations:    print_lst =print_lst+str(o) + '\n'
    
    
#     return  f'Will Recommend for:  {full_title}', output_str, recomendations
    return output_str, f'Recommendations for {full_title}: \n\n {print_lst}'

In [ ]:
# intf = gr.Interface(fn=search_movies,
#                     inputs=gr.Textbox(lines=1,placeholder="Put Your Favorite Movie Here To See Similar Movie Recommendations..."),
#                     outputs=["text","text"],
#                     examples=['Child\'s Play (1988)','Rushmore','brothers Bloom','call me by your name', 'Jumanji (1995)','The Three Amigos'],
#                     title="Dave's Fast.AI Movie Recommender",
#                     article='This recommender uses a collaborative filtering model fashioned from the [Fast.AI](https://github.com/fastai/fastbook/blob/master/08_collab.ipynb) library and trained on the 9 million movie reviews from the [Movie Lense Dataset](https://grouplens.org/datasets/movielens/25m/).  It trains Latent Factors to give movies and users descriptive weights that help predict thier reivews based on these factors.  The model was trained completely WITHOUT titles, descriptions, or categories and amazingly these things can be added to imporove this model.  Enjoy!'                    
# )
# # intf.launch(
# #     inline=False, #True publishes the page in notebook
# #     share=False) #True temporarily publishes the app on gradio website for 72 hours

In [ ]:
# movie_input = 'Buffalo Soldiers (2001)'
# ft = search_movies(movie_input)

In [ ]:
# get_movie_recs(movie_input)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()